In [1]:
import tensorflow as tf
print(tf.__version__)

2.1.0


In [6]:
import copy
import math
import os

import pandas as pd
import numpy as np
import tensorflow.keras.layers as L
from tensorflow.keras import Model
from sklearn.metrics import f1_score
from tensorflow.keras import callbacks
from tensorflow.keras.callbacks import Callback, LearningRateScheduler, EarlyStopping, ModelCheckpoint
import math
# from tqdm.notebook import tqdm
import gc
import os
from sklearn.metrics import recall_score, f1_score, precision_score
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.layers import Layer
from tensorflow.keras import layers
from tensorflow.keras.layers import Activation
from tensorflow.keras.utils import get_custom_objects
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from sklearn.model_selection import KFold
from tensorflow.python.keras.engine import training

LEN_ = 4000
n_epoch = 200
batch_size = 32
STEP_ = LEN_
folds = 5
learning_rate = 0.001
SEED_ = 987654321
patience = 20
FOLD_ = 0

lr_schedule = LearningRateScheduler(
    PolynomialDecay(
      initial_learning_rate = learning_rate,
      decay_steps = n_epoch,
      end_learning_rate = 0.0
    )
)

early_stopping = EarlyStopping(patience = patience, verbose=0)


In [7]:
class Mish(Activation):
    '''
    Mish Activation Function.
    .. math::
        mish(x) = x * tanh(softplus(x)) = x * tanh(ln(1 + e^{x}))
    Shape:
        - Input: Arbitrary. Use the keyword argument `input_shape`
        (tuple of integers, does not include the samples axis)
        when using this layer as the first layer in a model.
        - Output: Same shape as the input.
    Examples:
        >>> X = Activation('Mish', name="conv1_act")(X_input)
    '''

    def __init__(self, activation, **kwargs):
        super(Mish, self).__init__(activation, **kwargs)
        self.__name__ = 'Mish'


def mish(inputs):
    return inputs * tf.math.tanh(tf.math.softplus(inputs))

get_custom_objects().update({'Mish': Mish(mish)})

class macroF1(Callback):
    def __init__(self, model, inputs, targets):
        self.model = model
        self.inputs = inputs
        self.targets = targets.reshape(-1)

    def on_epoch_end(self, epoch, logs):
        pred = np.argmax(self.model.predict(self.inputs), axis=2).reshape(-1)
        f1_val = f1_score(self.targets, pred, average="macro")
        print("  --  val_f1_macro_score: ", f1_val)
        

def load_trained_model(weights_path):
    modelf = Unet_model()
    modelf.load_weights(weights_path)
    
    return(modelf)



In [8]:
DEFAULT_BLOCKS_ARGS = [{
    'kernel_size': 3,
    'repeats': 1,
    'filters_in': 32,
    'filters_out': 16,
    'expand_ratio': 1,
    'id_skip': True,
    'strides': 1,
    'se_ratio': 0.25
}, {
    'kernel_size': 3,
    'repeats': 2,
    'filters_in': 16,
    'filters_out': 24,
    'expand_ratio': 6,
    'id_skip': True,
    'strides': 2,
    'se_ratio': 0.25
}, {
    'kernel_size': 5,
    'repeats': 2,
    'filters_in': 24,
    'filters_out': 40,
    'expand_ratio': 6,
    'id_skip': True,
    'strides': 2,
    'se_ratio': 0.25
}, {
    'kernel_size': 3,
    'repeats': 3,
    'filters_in': 40,
    'filters_out': 80,
    'expand_ratio': 6,
    'id_skip': True,
    'strides': 2,
    'se_ratio': 0.25
}, {
    'kernel_size': 5,
    'repeats': 3,
    'filters_in': 80,
    'filters_out': 112,
    'expand_ratio': 6,
    'id_skip': True,
    'strides': 1,
    'se_ratio': 0.25
}, {
    'kernel_size': 5,
    'repeats': 4,
    'filters_in': 112,
    'filters_out': 192,
    'expand_ratio': 6,
    'id_skip': True,
    'strides': 2,
    'se_ratio': 0.25
}, {
    'kernel_size': 3,
    'repeats': 1,
    'filters_in': 192,
    'filters_out': 320,
    'expand_ratio': 6,
    'id_skip': True,
    'strides': 1,
    'se_ratio': 0.25
}]

CONV_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 2.0,
        'mode': 'fan_out',
        'distribution': 'truncated_normal'
    }
}

DENSE_KERNEL_INITIALIZER = {
    'class_name': 'VarianceScaling',
    'config': {
        'scale': 1. / 3.,
        'mode': 'fan_out',
        'distribution': 'uniform'
    }
}


def EfficientNet(
    width_coefficient,
    depth_coefficient,
    dropout_rate = 0.2,
    drop_connect_rate = 0.2,
    depth_divisor = 8,
    activation = 'Mish',
    blocks_args = 'default',
    model_name = 'efficientnet',
    include_top = True,
    input_shape = None,
    pooling = None,
    classes = 11,
    classifier_activation='softmax'):

  """Instantiates the EfficientNet architecture using given scaling coefficients.
  Reference paper:
  - [EfficientNet: Rethinking Model Scaling for Convolutional Neural Networks]
    (https://arxiv.org/abs/1905.11946) (ICML 2019)
  Optionally loads weights pre-trained on ImageNet.
  Note that the data format convention used by the model is
  the one specified in your Keras config at `~/.keras/keras.json`.
  Arguments:
    width_coefficient: float, scaling coefficient for network width.
    depth_coefficient: float, scaling coefficient for network depth.
    default_size: integer, default input image size.
    dropout_rate: float, dropout rate before final classifier layer.
    drop_connect_rate: float, dropout rate at skip connections.
    depth_divisor: integer, a unit of network width.
    activation: activation function.
    blocks_args: list of dicts, parameters to construct block modules.
    model_name: string, model name.
    include_top: whether to include the fully-connected
        layer at the top of the network.
    weights: one of `None` (random initialization),
          'imagenet' (pre-training on ImageNet),
          or the path to the weights file to be loaded.
    input_tensor: optional Keras tensor
        (i.e. output of `layers.Input()`)
        to use as image input for the model.
    input_shape: optional shape tuple, only to be specified
        if `include_top` is False.
        It should have exactly 3 inputs channels.
    pooling: optional pooling mode for feature extraction
        when `include_top` is `False`.
        - `None` means that the output of the model will be
            the 4D tensor output of the
            last convolutional layer.
        - `avg` means that global average pooling
            will be applied to the output of the
            last convolutional layer, and thus
            the output of the model will be a 2D tensor.
        - `max` means that global max pooling will
            be applied.
    classes: optional number of classes to classify images
        into, only to be specified if `include_top` is True, and
        if no `weights` argument is specified.
    classifier_activation: A `str` or callable. The activation function to use
        on the "top" layer. Ignored unless `include_top=True`. Set
        `classifier_activation=None` to return the logits of the "top" layer.
  Returns:
    A `keras.Model` instance.
  Raises:
    ValueError: in case of invalid argument for `weights`,
      or invalid input shape.
    ValueError: if `classifier_activation` is not `softmax` or `None` when
      using a pretrained top layer.
  """

  if blocks_args == 'default':
    blocks_args = DEFAULT_BLOCKS_ARGS
   
  input_layer = layers.Input(shape=input_shape)

  bn_axis = 2

  def round_filters(filters, divisor=depth_divisor):
    """Round number of filters based on depth multiplier."""
    filters *= width_coefficient
    new_filters = max(divisor, int(filters + divisor / 2) // divisor * divisor)
    # Make sure that round down does not go down by more than 10%.
    if new_filters < 0.9 * filters:
      new_filters += divisor
    return int(new_filters)

  def round_repeats(repeats):
    """Round number of repeats based on depth multiplier."""
    return int(math.ceil(depth_coefficient * repeats))

  # Build stem
  x = input_layer
  x = layers.Conv1D(
      round_filters(32),
      3,
      strides=2,
      padding='valid',
      use_bias=False,
      kernel_initializer=CONV_KERNEL_INITIALIZER,
      name='stem_conv')(x)
  x = layers.BatchNormalization(axis=bn_axis, name='stem_bn')(x)
  x = layers.Activation(activation, name='stem_activation')(x)

  # Build blocks
  blocks_args = copy.deepcopy(blocks_args)

  b = 0
  blocks = float(sum(args['repeats'] for args in blocks_args))
  for (i, args) in enumerate(blocks_args):
    assert args['repeats'] > 0
    # Update block input and output filters based on depth multiplier.
    args['filters_in'] = round_filters(args['filters_in'])
    args['filters_out'] = round_filters(args['filters_out'])

    for j in range(round_repeats(args.pop('repeats'))):
      # The first block needs to take care of stride and filter size increase.
      if j > 0:
        args['strides'] = 1
        args['filters_in'] = args['filters_out']
      x = block(
          x,
          activation,
          drop_connect_rate * b / blocks,
          name='block{}{}_'.format(i + 1, chr(j + 97)),
          **args)
      b += 1

  # Build top
  x = layers.Conv1D(
      round_filters(1280),
      1,
      padding='same',
      use_bias=False,
      kernel_initializer=CONV_KERNEL_INITIALIZER,
      name='top_conv')(x)
  x = layers.BatchNormalization(axis=bn_axis, name='top_bn')(x)
  x = layers.Activation(activation, name='top_activation')(x)
  if include_top:
    x = layers.GlobalAveragePooling1D(name='avg_pool')(x)
    if dropout_rate > 0:
      x = layers.Dropout(dropout_rate, name='top_dropout')(x)
    x = layers.Dense(
        classes,
        activation=classifier_activation,
        kernel_initializer=DENSE_KERNEL_INITIALIZER,
        name='predictions')(x)
  else:
    if pooling == 'avg':
      x = layers.GlobalAveragePooling1D(name='avg_pool')(x)
    elif pooling == 'max':
      x = layers.GlobalMaxPooling1D(name='max_pool')(x)

  # Create model.
  model = training.Model(input_layer, x, name=model_name)

  return model


def block(inputs,
          activation='Mish',
          drop_rate=0.,
          name='',
          filters_in=32,
          filters_out=16,
          kernel_size=3,
          strides=1,
          expand_ratio=1,
          se_ratio=0.,
          id_skip=True):
  """An inverted residual block.
  Arguments:
      inputs: input tensor.
      activation: activation function.
      drop_rate: float between 0 and 1, fraction of the input units to drop.
      name: string, block label.
      filters_in: integer, the number of input filters.
      filters_out: integer, the number of output filters.
      kernel_size: integer, the dimension of the convolution window.
      strides: integer, the stride of the convolution.
      expand_ratio: integer, scaling coefficient for the input filters.
      se_ratio: float between 0 and 1, fraction to squeeze the input filters.
      id_skip: boolean.
  Returns:
      output tensor for the block.
  """
  bn_axis = 2
  conv_pad = 'valid'

  # Expansion phase
  filters = filters_in * expand_ratio
  if expand_ratio != 1:
    x = layers.Conv1D(
        filters,
        1,
        padding='same',
        use_bias=False,
        kernel_initializer=CONV_KERNEL_INITIALIZER,
        name=name + 'expand_conv')(
            inputs)
    x = layers.BatchNormalization(axis=bn_axis, name=name + 'expand_bn')(x)
    x = layers.Activation(activation, name=name + 'expand_activation')(x)
  else:
    x = inputs

  x = layers.SeparableConv1D(
      filters,
      kernel_size,
      strides=strides,
      padding='same',
      use_bias=False,
      kernel_initializer=CONV_KERNEL_INITIALIZER,
      name=name + 'dwconv')(x)
  x = layers.BatchNormalization(axis=bn_axis, name=name + 'bn')(x)
  x = layers.Activation(activation, name=name + 'activation')(x)

  # Squeeze and Excitation phase
  if 0 < se_ratio <= 1:
    filters_se = max(1, int(filters_in * se_ratio))
    se = layers.GlobalAveragePooling1D(name=name + 'se_squeeze')(x)
    se = layers.Reshape((1, filters), name=name + 'se_reshape')(se)
    se = layers.Conv1D(
        filters_se,
        1,
        padding='same',
        activation=activation,
        kernel_initializer=CONV_KERNEL_INITIALIZER,
        name=name + 'se_reduce')(
            se)
    se = layers.Conv1D(
        filters,
        1,
        padding='same',
        activation='sigmoid',
        kernel_initializer=CONV_KERNEL_INITIALIZER,
        name=name + 'se_expand')(se)
    x = layers.multiply([x, se], name=name + 'se_excite')

  # Output phase
  x = layers.Conv1D(
      filters_out,
      1,
      padding='same',
      use_bias=False,
      kernel_initializer=CONV_KERNEL_INITIALIZER,
      name=name + 'project_conv')(x)
  x = layers.BatchNormalization(axis=bn_axis, name=name + 'project_bn')(x)
  if id_skip and strides == 1 and filters_in == filters_out:
    if drop_rate > 0:
      x = layers.Dropout(
          drop_rate, noise_shape=(None, 1, 1), name=name + 'drop')(x)
    x = layers.add([x, inputs], name=name + 'add')
  return x



def EfficientNetB5(include_top=True,
                   input_tensor=None,
                   input_shape=None,
                   pooling=None,
                   classes=11,
                   **kwargs):
  return EfficientNet(
      1.6,
      2.2,
      456,
      0.4,
      model_name='efficientnetb5',
      include_top=include_top,
      input_shape=input_shape,
      pooling=pooling,
      classes=classes,
      **kwargs)


def EfficientNetB6(include_top=True,
                   input_shape=None,
                   pooling=None,
                   classes=11,
                   **kwargs):
  return EfficientNet(
      1.8,
      2.6,
      0.5,
      model_name='efficientnetb6',
      include_top=include_top,
      input_shape=input_shape,
      pooling=pooling,
      classes=classes,
      **kwargs)

def EfficientNetB0(include_top=True,
                   input_shape=None,
                   pooling=None,
                   classes=11,
                   **kwargs):
  return EfficientNet(
      1.0,
      1.0,
      0.2,
      model_name='efficientnetb0',
      include_top=include_top,
      input_shape=input_shape,
      pooling=pooling,
      classes=classes,
      **kwargs)

def EfficientNetB7(include_top=True,
                   input_shape=None,
                   pooling=None,
                   classes=11,
                   **kwargs):
    
    return EfficientNet(
      2.0,
      3.1,
      0.5,
      model_name='efficientnetb7',
      include_top=include_top,
      input_shape=input_shape,
      pooling=pooling,
      classes=classes,
      **kwargs)


In [32]:
def Conv2dBn(
        filters,
        kernel_size,
        activation='Mish',
        strides = 1,
        kernel_initializer='he_uniform',
        padding='valid'):
    """Extension of Conv2D layer with batchnorm"""

    bn_axis = 2

    def wrapper(input_tensor):

        x = layers.Conv1D(
              filters,
              kernel_size,
              strides=strides,
              padding=padding)(input_tensor)
        x = layers.BatchNormalization(axis=bn_axis, name='stem_bn')(x)
        x = layers.Activation(activation, name='stem_activation')(x)

        return x

    return wrapper

def Conv3x3BnMish(filters, use_batchnorm, name=None):

    def wrapper(input_tensor):
        return Conv2dBn(
            filters,
            kernel_size=3,
            activation='Mish',
            kernel_initializer='he_uniform',
            padding='same',
        )(input_tensor)

    return wrapper


def DecoderUpsamplingX2Block(filters, stage, use_batchnorm=False):
    up_name = 'decoder_stage{}_upsampling'.format(stage)
    conv1_name = 'decoder_stage{}a'.format(stage)
    conv2_name = 'decoder_stage{}b'.format(stage)
    concat_name = 'decoder_stage{}_concat'.format(stage)

    concat_axis = 2
    
    def wrapper(input_tensor, skip=None):
        x = layers.UpSampling1D(size=2, name=up_name)(input_tensor)

        if skip is not None:
            x = layers.Concatenate(axis=concat_axis, name=concat_name)([x, skip])

        x = Conv3x3BnMish(filters, use_batchnorm, name=conv1_name)(x)
        x = Conv3x3BnMish(filters, use_batchnorm, name=conv2_name)(x)

        return x

    return wrapper

feature_layer = ('block6a_expand_activation', 'block4a_expand_activation',
                   'block3a_expand_activation', 'block2a_expand_activation')

# ---------------------------------------------------------------------
#  Unet Decoder
# ---------------------------------------------------------------------

def build_unet(
        backbone,
        decoder_block = DecoderUpsamplingX2Block,
        skip_connection_layers = feature_layer,
        decoder_filters = (256, 128, 64, 32, 16),
        n_upsample_blocks=5,
        classes=1,
        activation='sigmoid',
        use_batchnorm=True,
):
    input_ = backbone.input
    x = backbone.output

    # extract skip connections
    skips = ([backbone.get_layer(name=i).output for i in skip_connection_layers])

    # add center block if previous operation was maxpooling (for vgg models)
    if isinstance(backbone.layers[-1], layers.MaxPooling1D):
        x = Conv3x3BnReLU(512, use_batchnorm, name='center_block1')(x)
        x = Conv3x3BnReLU(512, use_batchnorm, name='center_block2')(x)

    # building decoder blocks
    for i in range(n_upsample_blocks):

        if i < len(skips):
            skip = skips[i]
        else:
            skip = None

        x = decoder_block(decoder_filters[i], stage=i, use_batchnorm=use_batchnorm)(x, skip)

    # model head (define number of output classes)
    x = layers.Conv1D(
        filters=classes,
        kernel_size=(3, 3),
        padding='same',
        use_bias=True,
        kernel_initializer='glorot_uniform',
        name='final_conv',
    )(x)
    x = layers.Activation(activation, name=activation)(x)

    # create keras model instance
    model = models.Model(input_, x)

    return model


In [33]:
# backbone = EfficientNetB0(include_top=False, input_shape = (LEN_, 1))
model = build_unet(backbone)

ValueError: A `Concatenate` layer requires inputs with matching shapes except for the concat axis. Got inputs shapes: [(None, 2000, 64), (None, 1999, 96)]

In [10]:
model.summary()

Model: "efficientnetb0"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 4000, 1)]    0                                            
__________________________________________________________________________________________________
stem_conv (Conv1D)              (None, 1999, 32)     96          input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 1999, 32)     128         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 1999, 32)     0           stem_bn[0][0]                    
_____________________________________________________________________________________

In [ ]:
train = pd.read_csv("../input/clean-removal-of-data-drift/train.csv")
_ = pd.read_csv("../input/liverpool-ion-switching/train.csv")
train['time'] = _['time']
del _

test = pd.read_csv("../input/clean-removal-of-data-drift/test.csv")
_ = pd.read_csv("../input/liverpool-ion-switching/test.csv")
test['time'] = _['time']
del _

n_classes = train.channel.unique().shape[0]


train_mean = train.signal.mean()
train_sigma = train.signal.std()

train['signal'] = (train['signal'] - train_mean)/train_sigma
test['signal'] = (test['signal'] - train_mean)/train_sigma

gc.collect()

In [ ]:
assert train.shape[0] % LEN_ == 0, 'Check dimension of LEN_ in comparison to size of train. It need to be a multiple!'

train_signal = train['signal'].values.reshape(-1, LEN_, 1)
train_target = train['channel'].values.reshape(-1, LEN_, 1)

test_signal = test["signal"].values.reshape(-1,LEN_,1)


In [ ]:
EfficientNetB0

In [ ]:
efficientnet = EfficientNetB0(weights = 'imagenet', include_top = False, classes = n_classes, input_shape = input_shape)


In [ ]:
%%time
kf = KFold(n_splits = folds, shuffle = True, random_state = SEED_)

for fold_n, (train_index, valid_index) in enumerate(kf.split(train_signal, train_target)):
    print(f'BEGIN FOLD:     {fold_n} -------\n\n\n')
    
    model = Unet_model()
    model_checkpoint = ModelCheckpoint("model" + str(fold_n) + ".hdf5",
                                       save_best_only=True, verbose=0, monitor='val_loss', mode='min')

    X_train, X_valid = train_signal[train_index,:,:], train_signal[valid_index,:,:]
    y_train, y_valid = train_target[train_index,:,:], train_target[valid_index,:,:]

    callBacks = [early_stopping, model_checkpoint, lr_schedule, macroF1(model, X_valid, y_valid)]

    history = model.fit(
        X_train, y_train,
        epochs = n_epoch,
        callbacks = callBacks,
        verbose = 1,
        validation_data = (X_valid, y_valid)
    )
    print(f'\n\n\nENDED FOLD:     {fold_n} -------\n\n\n')

In [ ]:
%%time
pred = np.zeros((test_signal.shape[0], test_signal.shape[1], n_classes))
for fold_ in range(folds):
    model = load_trained_model("model" + str(fold_) + ".hdf5")
    pred += model.predict(test_signal)/folds


np.save("pred.npy", pred, allow_pickle = True)

In [ ]:
sub = pd.read_csv("../input/liverpool-ion-switching/sample_submission.csv", dtype={'time':str})

sub['open_channels'] = np.argmax(pred, axis = 2).reshape(-1)
sub.to_csv('submission.csv', index=False)